In [103]:
crawling_list = ['물건명', '분양가', '매매가', '소유자', '계약면적', '전용면적', '융자금', '특징', '중개업소',
                 '월관리비', '방수', '욕실수', '입주가능일', '신축일', '보증금', '월세', '중개보수', '공과금']

In [105]:
'''
오피스텔 : rletTypeCd=A02
매매 : tradeTypeCd=A1
Highest Year of School Completed : hscpTypeCd=A02
법정동 코드 API http://juso.seoul.go.kr/openapi/helps/SearchApi_jibun.aspx
http://www.code.go.kr/
https://financedata.github.io/posts/korea-area-code.html
1168010300 개포동
1168010800 논현동
1168010600 대치동
1168011800 도곡동
1168010500 삼성동
1168011100 세곡동
1168011500 수서동
1168010700 신사동
'''

'\n오피스텔 : rletTypeCd=A02\n매매 : tradeTypeCd=A1\nHighest Year of School Completed : hscpTypeCd=A02\n법정동 코드 API http://juso.seoul.go.kr/openapi/helps/SearchApi_jibun.aspx\nhttp://www.code.go.kr/ \n1168010300 개포동\n1168010800 논현동\n1168010600 대치동\n1168011800 도곡동\n1168010500 삼성동\n1168011100 세곡동\n1168011500 수서동\n1168010700 신사동\n'

In [117]:
from urllib import request as rq
from bs4 import BeautifulSoup as bs
from datetime import datetime
import pandas as pd

df_village_code = pd.read_csv('https://goo.gl/tM6r3v', sep='\t', dtype={'법정동코드':str})
df_village_code = df_village_code[df_village_code["폐지여부"] == "존재"]
print(df_village_code.shape)

(20544, 3)


In [133]:
# 동 list
village_lists = list(df_village_code['법정동명'].str.split(' '))
village_list = [x[-1] for x in village_lists]
village_list[:20]

['서울특별시',
 '종로구',
 '청운동',
 '신교동',
 '궁정동',
 '효자동',
 '창성동',
 '통의동',
 '적선동',
 '통인동',
 '누상동',
 '누하동',
 '옥인동',
 '체부동',
 '필운동',
 '내자동',
 '사직동',
 '도렴동',
 '당주동',
 '내수동']

In [135]:
# 도시 list
city_list = [x[0] for x in village_lists]
city_list[:20]

['서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시']

In [136]:
df_village_code["도시"] = city_list
df_village_code["동"] = village_list
df_village_code.head()

,법정동코드,법정동명,폐지여부,도시,동
0,1100000000,서울특별시,존재,서울특별시,서울특별시
1,1111000000,서울특별시 종로구,존재,서울특별시,종로구
2,1111010100,서울특별시 종로구 청운동,존재,서울특별시,청운동
3,1111010200,서울특별시 종로구 신교동,존재,서울특별시,신교동
4,1111010300,서울특별시 종로구 궁정동,존재,서울특별시,궁정동


In [140]:
df_village_code[df_village_code["동"] == "개포동"]['법정동코드'].values[0]

'1168010300'

In [ ]:
# gangnam_dict = {"개포동":1168010300, "논현동":1168010800}
# village = input("동을 입력해주세요.")
# village_code = gangnam_dict[village]



root_url = "https://land.naver.com/"
searching_url_list = []

# basic_url = 'https://land.naver.com/article/articleList.nhn?rletTypeCd=A02&tradeTypeCd=A1&hscpTypeCd=A02&cortarNo=%d'%village_code
basic_url = 'https://land.naver.com/article/articleList.nhn?rletTypeCd=A02&tradeTypeCd=A1&hscpTypeCd=A02&cortarNo=1168010300'
basic_html = rq.urlopen(basic_url).read()
basic_soup = bs(basic_html, "html.parser")
basic_elems = basic_soup.find_all("span", {"class":"btn_naverlink"})
for i in range(len(basic_elems)):
    estate_url = root_url + basic_elems[i].find("a").attrs["href"]
    searching_url_list.append(estate_url)
    
searching_url_list


In [3]:
searching_url = searching_url_list[0]
searching_html = rq.urlopen(searching_url).read()
searching_soup = bs(searching_html, "html.parser")

# title
title_elem = searching_soup.find("h2", {"class":"t_adr ellipsis"})
title = title_elem.text

# get price
rate_basic = searching_soup.find_all("p", {"class":"rate_info"})
price = rate_basic[1].find_all("span")[0].text
price = int(price.split("만")[0].replace(",",""))
print(title, price)

대청타워 1동 20000


In [4]:
# get contract area
areas = ""
area = rate_basic[0].find_all("span")
for i in range(len(area)):
    areas += area[i].text
    
areas_list = areas.split('/')
contract_area = float(areas_list[0])

# get exclusive area
exclusive_area = float(areas_list[1].split('㎡')[0])

print(contract_area, exclusive_area)

55.2 32.44


In [5]:
# specific floor
summary_basic = searching_soup.find("div", {"class":"view_info"}).find_all("div", {"class":"inner"})
floors = summary_basic[1].text.split("/")
specific_floor = int(floors[0])

# total floor
total_floor = int(floors[1].split("층")[0])

print(specific_floor, total_floor)

6 26


In [6]:
# rooms
room_and_bath = summary_basic[3].text.split("/")
rooms = int(room_and_bath[0])

# baths
baths = int(room_and_bath[1].split('개')[0])

print(rooms, baths)

1 1


In [7]:
# loan amount
try:
    loan_amount = int(summary_basic[5].text.split("만")[0])
except ValueError as e:
    loan_amount = 0
    
print(loan_amount)

0


In [8]:
# moveable
try:
    moveable = summary_basic[7].text
except ValueError as e:
    moveable = "-"
    
print(moveable)

즉시입주


In [9]:
# administration cost
try:
    administration_cost = int(summary_basic[9].text.split("원")[0].strip().replace(",",""))
except ValueError as e:
    administration_cost = "-"
    
print(administration_cost)

100000


In [10]:
# deposit amount

deposit_and_rentfee = summary_basic[11].text.split("/")
try:    
    deposit = int(deposit_and_rentfee[0].strip())
except ValueError as e:
    deposit = "-"
    
# rent fee
try:
    rent_fee = int(deposit_and_rentfee[1].split("만")[0].strip())
except ValueError as e:
    rent_fee = "-"
    
print(deposit, rent_fee)    

500 65


In [11]:
# characteristics
try:
    characteristics = summary_basic[13].text
except ValueError as e:
    characteristics = "-"
characteristics

'탁트인공원전망 오래월세중 수익좋음'

In [12]:
# intermediary
try:
    intermediary = summary_basic[15].text
except ValueError as e:
    intermediary = "-"
    
print(intermediary)

나산공인중개사사무소 02-3413-7700 l 소재지 대표: 변동주 010-6207-3097 


In [66]:
# utility bills
tax_soup = searching_soup.find_all('ul', {"class":"lst_tax"})[1]
replaceable_chr_list = ['\\', 'n', 't', ',', ' ', '약', '원']

try:
    utility_bills = tax_soup.find("strong", {"class":"highlight"}).text

    for i in replaceable_chr_list:

        utility_bills = utility_bills.replace(i, '').strip()

    utility_bills = int(utility_bills)
    
except ValueError as e:
    utility_bills = "-"
    
utility_bills

9434000

In [74]:
# intermediate pay
intermediate_soup = searching_soup.find_all('ul', {"class":"lst_tax"})[0]
try:
    intermediate_pay = int(intermediate_soup.find("strong").text)
except ValueError as e:
    intermediate_pay = "-"
    
print(intermediate_pay)

100


In [101]:
# completion date
completion_elem = searching_soup.find("div", {"class":"div_detail"}).find_all("div", {"class":"inner"})

try:
    completion_date = datetime.strptime(completion_elem[-1].text, '%Y.%m.')
except ValueError as e:
    completion_date = "-"
    
print(completion_date.year, '/', completion_date.month)

1997 / 5
